In [99]:
from utils.testproblems import generate_random_prob
from utils.polycasebuilder import generate_random_polynomials, get_arg_mapping
from trash.inputresolver import reassigneq
from graph.graphutils import edges_to_Ein_Eout, sources
from torchengine import AnalyticalSet, EliminateAnalysis, Function, EliminateAnalysisMergeResiduals
from scipy.optimize import minimize
import sympy as sp
import numpy as np
import torch
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [90]:
n_eqs, n_vars, sparsity, seed1, seed2 = 10, 13, 1.1, 42, 12345 #1.7
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed1, sparsity)
polynomials, var_mapping, edges, tree = generate_random_polynomials(eqv, dout, n_eqs)
symb_mapping = get_arg_mapping(var_mapping, symbol_map=True)

indices = {idx: torch.tensor([idx]) for idx in symb_mapping.keys()}
sets = {}
for polyidx, poly in polynomials.items():
    outputvar = symb_mapping[edges[1][polyidx][0]]
    inputvars = [symb_mapping[elt] for elt in edges[0][polyidx]]
    analysis_function = sp.simplify(reassigneq(None, poly, outputvar))
    if isinstance(analysis_function, sp.Float):
        tensor = torch.tensor([float(analysis_function)], dtype=torch.float64)
        f = lambda : tensor
    else:
        f = sp.lambdify(inputvars, analysis_function, torch) 
    residual = sp.lambdify(inputvars+[outputvar], poly, torch)   
    tupl = (edges[0][polyidx]+edges[1][polyidx], residual)
    triplet = (edges[0][polyidx], edges[1][polyidx], f)
    sets[polyidx] = AnalyticalSet(triplet, indices, forceresidual=tupl)

In [91]:
inputids = sorted(sources(*edges_to_Ein_Eout(edges)))
inputvars = [var_mapping[elt][0] for elt in inputids]
fobj = sum([(elt-1)**2 for elt in inputvars])
objective = Function((inputids, sp.lambdify(inputvars, fobj, torch)), indices)

In [133]:
optim_funcs_MDF = EliminateAnalysis([sets[k].analysis for k in sorted(sets.keys())],
                                [objective, lambda x: torch.tensor([]), lambda x: torch.tensor([])])
equality_constraints_AAO = EliminateAnalysisMergeResiduals([],[sets[k].residual for k in sorted(sets.keys())])
optim_funcs_AAO = EliminateAnalysis([],
                                [objective, lambda x: torch.tensor([]), equality_constraints_AAO])

In [134]:
def generate_optim_functions(optim_funcs, solvefor_indices, x):
    def eval_all(y):
        x[solvefor_indices] = torch.from_numpy(y).to(x.dtype)
        objval, ineqval, eqval = optim_funcs(x)
        return objval.item(), -ineqval, -eqval
    
    def obj_function(y):
        objval, _, _ = eval_all(y)
        return objval

    def ineq_function(y=None):
        _, ineqval, _ = eval_all(y)
        return ineqval
    
    def eq_function(y=None):
        _, _, eqval = eval_all(y)
        return eqval

    xguess = x[solvefor_indices].numpy()
    
    return xguess, obj_function, ineq_function, eq_function

In [135]:
optim_indices_MDF = torch.cat([indices[elt] for elt in inputids])
bnds_problem_MDF = [(0,10) for elt in optim_indices_MDF]

optim_indices_AAO = torch.cat([indices[k] for k in sorted(indices.keys())])
bnds_problem_AAO = [(0,10) for elt in optim_indices_AAO]
#set random seed to be reproducible
torch.manual_seed(43)
x0 = torch.rand(n_vars, dtype=torch.float64)
xguess, obj_function, ineq_function, eq_function = generate_optim_functions(optim_funcs_AAO, optim_indices_AAO, x0)
constraints = [{'type': 'eq', 'fun': eq_function}]

In [137]:
minimize(obj_function, xguess, constraints=constraints, method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 962868.6890955853
       x: [-2.521e-01  9.823e+02 ... -1.086e-07  7.929e+00]
     nit: 93
     jac: [-2.508e+00  1.963e+03 ...  0.000e+00  0.000e+00]
    nfev: 1485
    njev: 93

In [143]:
x0.numpy()

array([-0.25, 982.26, 356.97, 1.95, -0.2, -0, 8.66, 6851.16, -11.9, 0.03,
       -8310.65, -0, 7.93])

In [153]:
sets[9].analysis(x0).numpy()-x0.numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])